# 快速入门 GPT-4 Vison

从历史上看，语言模型系统仅接受**文本**作为输入。但是单一的输入形式，限制了大模型的应用落地范围。

随着技术发展，OpenAI 开发的 GPT-4 Turbo with Vision（简称 GPT-4V）允许模型接收**图像**作为输入，并回答关于它们的问题。

📢注意，目前在 Assistants API 中使用 GPT-4 时还不支持图像输入。

## 使用 GPT-4V 识别线上图像（URL）

![image_sample](https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg)

In [2]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
  model="gpt-4-turbo",
  messages=[
    {
      "role": "user",
      "content": [
        {"type": "text", "text": "介绍下这幅图?"},
        {
          "type": "image_url",
          "image_url": {
            "url": "https://upload.wikimedia.org/wikipedia/commons/thumb/d/dd/Gfp-wisconsin-madison-the-nature-boardwalk.jpg/2560px-Gfp-wisconsin-madison-the-nature-boardwalk.jpg",
          },
        },
      ],
    }
  ],
  max_tokens=300,
)

print(response.choices[0])

Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='这幅图展示了一个美丽的自然风景。图中有一条木制的栈道穿过一片绿色的草地，引人向远处漫步。栈道两侧的草丛高密，看起来非常生机勃勃。天空是晴朗的，有几朵散布的白云，增添了一种平静而广阔的感觉。远处的树木和丰富的绿色植被与蓝天形成了鲜明的对比。\n\n整体上，这是一幅引人入胜的自然风光照片，适合用来展示大自然的宁静和美丽，也可能吸引人们去户外探索和放松心情。这种环境通常适合散步、思考或进行摄影，作为远离城市喧嚣的一片净土。', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))


In [3]:
response.choices[0].message.content

'这幅图展示了一个美丽的自然风景。图中有一条木制的栈道穿过一片绿色的草地，引人向远处漫步。栈道两侧的草丛高密，看起来非常生机勃勃。天空是晴朗的，有几朵散布的白云，增添了一种平静而广阔的感觉。远处的树木和丰富的绿色植被与蓝天形成了鲜明的对比。\n\n整体上，这是一幅引人入胜的自然风光照片，适合用来展示大自然的宁静和美丽，也可能吸引人们去户外探索和放松心情。这种环境通常适合散步、思考或进行摄影，作为远离城市喧嚣的一片净土。'

### 封装成一个函数 query_image_description

In [4]:
def query_image_description(url, prompt="介绍下这幅图?"):
    client = OpenAI()  # 初始化 OpenAI 客户端
    
    # 发送请求给 OpenAI 的聊天模型
    response = client.chat.completions.create(
        model="gpt-4-turbo",  # 指定使用的模型
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": url}},
                ],
            }
        ],
        max_tokens=300,
    )
    
    # 返回模型的响应
    return response.choices[0].message.content


### 调用函数测试

![meme_0](https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg)

In [5]:
image_url = "https://p6.itc.cn/q_70/images03/20200602/0c267a0d3d814c9783659eb956969ba1.jpeg"
content = query_image_description(image_url)
print(content)

这幅图展示了两种幽默的狗的插画：左侧是一只身体肌肉发达像人类的狗，被赋予了“16岁的我”（即想象中的自己）的标签；右侧是一只普通体态的狗，被称为“工作后的我”，看起来比较平常和疲惫。

图中左侧的狗插画带有夸张的人类肌肉，给人一种强壮和自信的形象，标签下的中文文字“我刚健一个光阴，身体素质高，成长质量好”意味着年轻时的自我理想化和健康形象。

右侧的狗看起来更接近实际的狗的样子，表情稍显懒散和疲惫，标签下的文字“好紧张然，好难熬疲劳，无法得不补充，能源快要没电，就要直接关机


### 使用 GPT-4V 识别本地图像文件（Base64编码）


In [6]:
from openai import OpenAI
import base64
import requests
import json

client = OpenAI()  # 初始化 OpenAI 客户端

def query_base64_image_description(image_path, prompt="解释下图里的内容？", max_tokens=1000):

    # 实现 Base64 编码
    def encode_image(path):
        with open(path, "rb") as image_file:
            return base64.b64encode(image_file.read()).decode('utf-8')

    # 获取图像的 Base64 编码字符串
    base64_image = encode_image(image_path)

    # 构造请求的 HTTP Header
    headers = {
        "Content-Type": "application/json",
        "Authorization": f"Bearer {client.api_key}"
    }

    # 构造请求的负载
    payload = {
        "model": "gpt-4-turbo",
        "messages": [
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{base64_image}"}}
                ]
            }
        ],
        "max_tokens": max_tokens
    }

    # 发送 HTTP 请求
    response = requests.post("https://api.openai.com/v1/chat/completions", headers=headers, json=payload)

    # 检查响应并提取所需的 content 字段
    if response.status_code == 200:
        response_data = response.json()
        content = response_data['choices'][0]['message']['content']
        return content
    else:
        return f"Error: {response.status_code}, {response.text}"

#### 使用 Assistants API生成的 GDP 40年对比曲线图

![gdp_data](./images/gdp_1980_2020.jpg)

In [7]:
content = query_base64_image_description("./images/gdp_1980_2020.jpg")
print(content)

这幅图表展示了1980年到2020年间美国、中国、日本和德国的国内生产总值（GDP）对比。横轴代表年份，纵轴代表GDP总量（单位是万亿美元）。

- 蓝色线代表美国，可以看出美国的GDP从1980年起持续上升，到2020年时超过20万亿美元。
- 红色线代表中国，其GDP在1990年代中期开始显著增长，2000年后增长速度更是显著，至2020年接近15万亿美元。
- 紫色线代表日本，其GDP在1990年代达到顶峰后有所波动，但整体呈平稳状态，在5万亿到6万亿美元之间。
- 绿色线代表德国，其GDP相对平稳，增长幅度较小，到2020年时约为4万亿美元。

此图表有效地展现了这四个国家经济增长的趋势和速度，尤其突出了中国的快速增长。


#### 使用 GPT-4V 识别手写体笔记

![](./images/handwriting_0.jpg)

In [8]:
content = query_base64_image_description("./images/handwriting_0.jpg")
print(content)

这张图片显示的是一本笔记本上的笔记，主要内容是关于自然语言处理（NLP）中的各种模型调优方法。这些笔记包括了以下几个部分：

1. **Prompt Tuning（提示调优）**：笔记提到了利用少量模型（例如FMT或small model）进行prompt tuning。其中，X代表输入的嵌入向量序列，X'是经过添加特殊token进行扩展后的序列，Y为输出，通过W矩阵和X'的乘积得到。

2. **Prefix Tuning（前缀调优）**：在这种调优策略中，W'是由特定的前缀Wp和原始的W拼接而成，以此来调整Transformer模型的Encoder/Decoder的行为。

3. **LoRA调权技术**：此部分介绍了LoRA（Low-Rank Adaptation）技术。这里使用的是低秩矩阵分解的方法来进行权重的调整，其中ΔW = AB，A和B是较低维度的矩阵，这使得可以在不显著增加模型大小的情况下调整模型权重。

4. **和LAMA-65以及QLoRA-4GB的提示性能比较数据**：这些内容可能是对比不同调优技术在特定数据集或性能指标上的表现。

笔记本上的文字是用中文书写的，涵盖了深入的技术细节和一些专业术语，指向深层次的技术讨论和研究。


#### 在 Jupyter 标准输出中渲染 Markdown 格式内容

In [9]:
from IPython.display import display, Markdown

# 使用 display 和 Markdown 函数显示 Markdown 内容
display(Markdown(content))

这张图片显示的是一本笔记本上的笔记，主要内容是关于自然语言处理（NLP）中的各种模型调优方法。这些笔记包括了以下几个部分：

1. **Prompt Tuning（提示调优）**：笔记提到了利用少量模型（例如FMT或small model）进行prompt tuning。其中，X代表输入的嵌入向量序列，X'是经过添加特殊token进行扩展后的序列，Y为输出，通过W矩阵和X'的乘积得到。

2. **Prefix Tuning（前缀调优）**：在这种调优策略中，W'是由特定的前缀Wp和原始的W拼接而成，以此来调整Transformer模型的Encoder/Decoder的行为。

3. **LoRA调权技术**：此部分介绍了LoRA（Low-Rank Adaptation）技术。这里使用的是低秩矩阵分解的方法来进行权重的调整，其中ΔW = AB，A和B是较低维度的矩阵，这使得可以在不显著增加模型大小的情况下调整模型权重。

4. **和LAMA-65以及QLoRA-4GB的提示性能比较数据**：这些内容可能是对比不同调优技术在特定数据集或性能指标上的表现。

笔记本上的文字是用中文书写的，涵盖了深入的技术细节和一些专业术语，指向深层次的技术讨论和研究。

![](./images/handwriting_1.jpg)

In [10]:
content = query_base64_image_description("./images/handwriting_1.jpg")
display(Markdown(content))

这张图片展示了一本笔记本的两页内容，主要记录了关于自然语言处理（NLP）领域中智能模型（尤其是Transformer模型）和调优方法的笔记。内容涉及现代机器学习技术的各种策略，特别是在大型语言模型（Large Language Models, LLMs）的调优上。

左页的内容主要聚焦在PEFT（可能是指某种模型调优方法的缩写或特定的学术术语）和各种基于prompt（提示）的调优方法。包括：

- Adapter、Prefix、Prompt Tuning等方法，这些都是不同的模型微调方法。
- 提到了一些文献或来源，例如Google、Stanford等。
- 记录了一系列相关技术和方法，如Soft Prompts、Instruction Tuning等。
- 细缀了相关的模型名称或技术，如LGPT、BLOOM、CLAiRE、ChatGLM等。

右页内容则覆盖了：

- 多模态指导微调（multi-modality instruction fine-tuning）的方法，如LLaMA（3B）。
- LORA、GLoRA等新策略和技术的名称。
- 对Adapters和各种调优方法（如Refine、MAM Adapters等）的进一步讨论和对比分析。
- 提到了更具体的调优技术，如scaled parallel adapters、MAM Adaptors等，并讨论了SOTA（State Of The Art）结果的改进。

这些笔记显然是为了记载和学习关于最新的NLP模型调优技术和方法的最新进展，适用于研究者或学生在学习或研究NLP及人工智能领域时使用。

## Homework: 


### #1

使用 GPT-4V 识别带有手写体文字的本地图像文件，分享结果。

### #2

整合 `query_base64_image_description` 函数和 Markdown 格式渲染方法，使得输出结果更易阅读。

In [13]:
def display_image_description(image_path, prompt="解释下图里的内容？"):
    """
    读取图像文件，获取描述并以Markdown格式显示结果
    
    Args:
        image_path: 图像文件路径
    """
    # 获取图像描述
    description = query_base64_image_description(image_path, prompt)
    
    # 构建Markdown格式的输出
    markdown_content = f"""
## 图像描述

![图像]({image_path})

### GPT-4V 识别结果：

{description}
"""
    
    # 显示格式化后的结果
    display(Markdown(markdown_content))

# 测试函数
# display_image_description("./images/handwriting_1.jpg")


In [15]:
display_image_description("./images/sam_handwriting.jpg", "识别图中文字。按照流程步骤，按照原意转述")


## 图像描述

![图像](./images/sam_handwriting.jpg)

### GPT-4V 识别结果：

图中的文字主要涉及一个流程或步骤的说明，带有多个方框、圆圈和箭头，显示了不同阶段和决策点。以下是图中各部分的内容：

1. 最顶端左侧写着：“传导”、“生活”、“D366”。
2. 右上角有一段数学表达式：“10...1+1=?”。
3. 中间部分有各种箭头连接的方框和圆圈，方框里写着“问题ID”，“问题简述”等。
4. 方框之间的连接箭头上有注释，例如“基础故障，故障分析”，“引导至知识库”。
5. 左侧部分有个大圆圈环绕着多个小圆圈，似乎表示一个校验或决策流程，圆圈中有“是”、“否”等决策选项。
6. 图中多处还有手写注释，如“全员check”、“发给相关同事”、“追踪反馈”，表明这可能是一个协作或问题解决的流程。

总体来看，该图似乎是某种事务处理或故障处理的流程图，涉及问题识别、分析及解决策略的多个步骤。


In [16]:
display_image_description("./images/sam_handwriting2.jpg", "解释下图里的内容？")


## 图像描述

![图像](./images/sam_handwriting2.jpg)

### GPT-4V 识别结果：

这张图片显示的是一张记录了多个日期和对应笔记的纸张。笔记主要涉及编程和软件开发的相关内容，例如Visual Studio使用、代码管理、项目文件探查等。内容有中英文混杂，说明记录者可能在使用中英文环境进行工作或学习。

- 2023年8月9日的记录提到了关于视图和控制器的提问和查找方法。
- 2023年8月10日的记录看似涉及了使用Visual Studio软件的问题，如如何区分自动生成代码与手动添加代码，如何在解决方案资源管理器中切换项目，以及如何映射和定位数据库表的列等。
- 2023年8月11日的记录进一步讨论了Visual Studio的文件和目录结构管理。

这些笔记可能是用于个人复习或作为现场开发会议的提点记录用。
